In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
poems = np.load('normalized_poem_text.npy')
raw_text = '\n'.join(poems)
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# int_to_char = dict((i, c) for i, c in enumerate(chars))

In [3]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Number of characters in the datatset: ", n_chars)
print("unique charachter count: ", n_vocab)

Number of characters in the datatset:  46468836
unique charachter count:  40


In [5]:
%%time
seq_length = 42 # play with this number
step_size = 1 # and this number
dataX = [] # list of sentences of length seq_length
dataY = [] # list of charachters following that sentence
for i in range(0, n_chars - seq_length, step_size):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_seq = len(dataX)
print("number of sequences: ", n_seq)

number of sequences:  46468794
Wall time: 7min 39s


In [6]:
dataX = np.array(dataX)
dataY = np.array(dataY)

In [8]:
# reshape X to be [samples, time steps, features]
# then normalize it
chunked_dataX = np.array_split(dataX, 114)
for chunk in chunked_dataX:
    chunk = np.reshape(chunk, (int(n_seq/114), seq_length, 1))
    chunk = chunk / float(n_vocab)

X = np.concatenate(chunked_dataX, axis=0)
X = np.reshape(X, (n_seq, seq_length, 1))

In [7]:
y = np_utils.to_categorical(dataY)

In [8]:
# save so you don't have to go through that again
np.save('input_sequences', X)
np.save('output_sequences', y)

# checkpoint


In [4]:
# load files
X = np.load('input_sequences.npy')
y = np.load('output_sequences.npy')

In [9]:
# build the LSTM model
# model = Sequential()
# model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))

# model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
from keras.optimizers import RMSprop
model = Sequential()
model.add(Dropout(0.3, input_shape=(X.shape[1], X.shape[2])))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))

optimizer = RMSprop(lr=0.01) # TODO: experiment with dropout and learning rates

model.compile(loss='categorical_crossentropy', optimizer= optimizer)

In [10]:
# define the checkpoint
w_path="weights/weights-epoch-{epoch:02d}-loss-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(w_path, monitor='loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')

In [ ]:
model.fit(X, y, epochs=20, batch_size=512, callbacks=[checkpoint])

Epoch 1/20
46468794/46468794 [==============================] - 13537s 291us/step - loss: 3.0450
Epoch 2/20
46468794/46468794 [==============================] - 13514s 291us/step - loss: 2.8399
Epoch 3/20
46468794/46468794 [==============================] - 13524s 291us/step - loss: 2.8597
Epoch 4/20
46468794/46468794 [==============================] - 13522s 291us/step - loss: 2.9085
Epoch 5/20
46468794/46468794 [==============================] - 13433s 289us/step - loss: 2.9566
Epoch 6/20
20916224/46468794 [============>.................] - ETA: 2:02:25 - loss: 2.9625

## Generate Text

In [ ]:

chars = # TODO: save char list externally and load it here
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
filename = ''

model = Sequential()
model.add(Dropout(0.3, input_shape=(X.shape[1], X.shape[2])))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))

model.load_weights(filename)
optimizer = RMSprop(lr=0.01) # TODO: experiment with dropout and learning rates
model.compile(loss='categorical_crossentropy', optimizer= optimizer)

In [ ]:
seed = ''
sequence = [char_to_int[char] for char in seed]
gen_text = ''
print('Seed:\n')
print(seed)
# generate characters
for i in range(140):
    x = np.reshape(sequence, (1, len(sequence), 1))
    x = x / len(chars)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction) # returns the index of the predicted char
    gen_text += int_to_char[index]
    sequence.append(index)
    sequence = sequence[1:len(sequence)]

print(gen_text)
print('Done.')